In [1]:
1+1

2

In [5]:
import os
from dotenv import load_dotenv
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.pregel import NodeBuilder

load_dotenv()


class ChatState(TypedDict):
    question: str
    answer: str


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, verbose=True)

# 인메모리 캐쉬를 위해 MemorySaver를 사용합니다.
memory = MemorySaver()


# 노드 정의
def generate_answer(state: ChatState) -> dict:
    """LLM을 호출하여 답변을 생성하는 노드 함수"""
    print("\n>>>>> LLM 노드 실행! (이 메시지는 캐쉬되지 않았을 때만 보입니다) <<<<<")
    question = state["question"]
    response = llm.invoke(question)
    return {"answer": response.content}


# 캐쉬 함수 정의
def get_cache_key(state: ChatState, config: dict) -> str:
    return state["question"]


# 노드 빌더를 이용하여 Cache 기능을 갖는 노드를 정의함
# cached_node_builder = NodeBuilder(generate_answer).cache(get_cache_key)

# StateGraph 빌더를 생성합니다.
builder = StateGraph(ChatState)

# NodeBuilder로 생성한 캐쉬 적용 노드를 그래프에 추가합니다.
builder.add_node("generate_answer_node", generate_answer)

# 엣지 추가: 그래프 흐름 정의
builder.add_edge(START, "generate_answer_node")  # 시작 -> 답변 생성 노드
builder.add_edge("generate_answer_node", END)  # 답변 생성 노드 -> 종료

# Checkpointer(캐쉬)를 연결하여 그래프를 컴파일합니다.
graph = builder.compile(checkpointer=memory)

In [6]:
# ---  그래프 실행 및 캐싱 확인 ---
# 대화 스레드를 식별하기 위한 설정
config = {"configurable": {"thread_id": "chat-thread-1"}}
question = "LangGraph에서 노드 캐싱은 어떻게 하나요?"
initial_state = {"question": question, "answer": ""}

# 첫 번째 호출: LLM 노드가 실행됩니다.
print("--- 첫 번째 질문 ---")
print(f"질문: {question}")
final_state = graph.invoke(initial_state, config)
print(f"답변: {final_state['answer']}")

# 두 번째 호출: 동일한 질문이므로 캐쉬된 결과를 반환합니다.
print("\n\n--- 두 번째 질문 (동일한 내용) ---")
print(f"질문: {question}")
# 'LLM 노드 실행!' 메시지가 출력되지 않아야 합니다.
cached_state = graph.invoke(initial_state, config)
print(f"답변: {cached_state['answer']}")

# 세 번째 호출: 다른 질문이므로 LLM 노드가 다시 실행됩니다.
print("\n\n--- 세 번째 질문 (다른 내용) ---")
new_question = "LangChain의 장점은 무엇인가요?"
new_initial_state = {"question": new_question, "answer": ""}
print(f"질문: {new_question}")
new_final_state = graph.invoke(new_initial_state, config)
print(f"답변: {new_final_state['answer']}")

--- 첫 번째 질문 ---
질문: LangGraph에서 노드 캐싱은 어떻게 하나요?

>>>>> LLM 노드 실행! (이 메시지는 캐쉬되지 않았을 때만 보입니다) <<<<<
답변: LangGraph에서 노드 캐싱을 구현하는 방법은 여러 가지가 있을 수 있지만, 일반적으로 다음과 같은 접근 방식을 사용할 수 있습니다.

1. **메모리 캐싱**: 노드를 메모리에 저장하여 빠르게 접근할 수 있도록 합니다. 예를 들어, Python의 딕셔너리나 리스트를 사용하여 노드를 저장하고, 필요할 때마다 조회합니다.

2. **디스크 캐싱**: 노드 데이터를 파일 시스템에 저장하여, 메모리 사용을 줄이고, 애플리케이션이 재시작되더라도 데이터를 유지할 수 있도록 합니다. JSON, CSV, 또는 데이터베이스를 사용할 수 있습니다.

3. **캐시 라이브러리 사용**: Redis, Memcached와 같은 외부 캐시 시스템을 사용하여 노드를 저장하고 관리할 수 있습니다. 이러한 시스템은 분산 환경에서도 유용하게 사용할 수 있습니다.

4. **TTL (Time to Live)**: 캐시된 노드에 TTL을 설정하여 일정 시간이 지나면 자동으로 삭제되도록 할 수 있습니다. 이를 통해 오래된 데이터를 자동으로 정리할 수 있습니다.

5. **변경 감지**: 노드의 변경 사항을 감지하여 캐시를 업데이트하는 방법도 있습니다. 예를 들어, 노드가 수정되면 해당 노드를 캐시에서 제거하거나 업데이트합니다.

구체적인 구현 방법은 사용하는 프로그래밍 언어와 프레임워크에 따라 다를 수 있으므로, LangGraph의 문서나 예제를 참조하여 적절한 방법을 선택하는 것이 좋습니다.


--- 두 번째 질문 (동일한 내용) ---
질문: LangGraph에서 노드 캐싱은 어떻게 하나요?

>>>>> LLM 노드 실행! (이 메시지는 캐쉬되지 않았을 때만 보입니다) <<<<<
답변: LangGraph에서 노드 캐싱을 구현하는 방법은 여러 가지가 있을 수 있지만, 일반적으로 다음과 같은 접근 방식을 사용할